# 네이버 지식인 크롤링

In [2]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from urllib.parse import quote
from tqdm import tqdm
from datetime import datetime, timedelta

## 날짜 입력
- 며칠동안의 데이터를 수집할지 입력
- 입력값: 3 -> 3일전 ~ 오늘날짜

In [3]:
KEYWORD = '대선후보'
N = 3
start_date = datetime.today() - timedelta(N)
FROM_DATE = start_date.strftime("%Y.%m.%d.")
TO_DATE = datetime.today().strftime("%Y.%m.%d.")

In [7]:
print(FROM_DATE, TO_DATE)

2021.04.17. 2021.04.20.


In [8]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
base_url = "https://www.naver.com/"
driver.get(base_url)
time.sleep(1)

# 지식iN 으로 이동
nav = driver.find_element_by_id('NM_FAVORITE')
lis = nav.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
lis[3].click()

# 키워드 검색 후 최신순 정렬
search_area = driver.find_element_by_class_name('search_area')
search_area.find_element_by_tag_name('input').send_keys(KEYWORD)    # 키워드 입력
driver.find_element_by_class_name('search_btn').click()     # 검색
driver.find_element_by_class_name('option_sort').find_elements_by_tag_name('li')[1].click()     # '최신순' 클릭
time.sleep(1)

# 기간 검색: FROM_DATE ~ TO_DATE
driver.find_element_by_id('au_date_select').click()
time.sleep(1)
from_date = driver.find_element_by_id('sel_from_date')
to_date = driver.find_element_by_id('sel_to_date')
# 시작날짜, 완료날짜 입력
actions = ActionChains(driver).double_click(from_date).send_keys(FROM_DATE).double_click(to_date).send_keys(TO_DATE)
actions.perform()
driver.find_element_by_xpath('/html/body/div[3]/div/ul/li[4]/input[3]').click()

# 전체 페이지수 수집
while True:
    pagination = driver.find_element_by_class_name('s_paging')
    pages = pagination.find_elements_by_tag_name('a')
    tmp = []
    for page in pages:
        tmp.append(page.text)
    if '다음페이지' in tmp:
        pages[-1].click()
        time.sleep(2)
    else:
        NUM_OF_PAGE = int(pagination.text.split()[-1])
        break
print("전체 페이지 수: ", NUM_OF_PAGE)
driver.close()

전체 페이지 수:  3


In [9]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
ans_url_list = []; total_ans = 0

# 질문들의 url 수집
for page in range(NUM_OF_PAGE):
    kin_url = f'https://kin.naver.com/search/list.nhn?sort=date&query={quote(KEYWORD)}'
    period_url = f'&period={FROM_DATE}%7C{TO_DATE}&section=kin'
    page_url = f'&page={page+1}'
    url = kin_url + period_url + page_url
    driver.get(url)
    time.sleep(1)
    #print(f"{page+1} 페이지")

    ul = driver.find_element_by_xpath('//*[@id="s_content"]/div[3]/ul')
    lis = ul.find_elements_by_tag_name('li')
    for li in lis:
        a_tag = li.find_element_by_tag_name('a')
        ans_url_list.append(a_tag.get_attribute('href'))

print(f'총 {len(ans_url_list)}개의 질문')

# 각 질문의 답변 개수
for i, ans_url in enumerate(ans_url_list):
    driver.get(ans_url)
    time.sleep(0.5)
    ans_cnt = int(driver.find_element_by_class_name('_answerCount.num').text)
    total_ans += ans_cnt
    print(f'{i+1}번째 : {ans_cnt}')
print("----------------------------------")
print(f'Total 답변 개수: {total_ans}')

driver.close()

총 21개의 질문
1번째 : 1
2번째 : 2
3번째 : 5
4번째 : 1
5번째 : 1
6번째 : 3
7번째 : 3
8번째 : 3
9번째 : 5
10번째 : 3
11번째 : 1
12번째 : 2
13번째 : 4
14번째 : 1
15번째 : 6
16번째 : 6
17번째 : 1
18번째 : 3
19번째 : 5
20번째 : 2
21번째 : 12
----------------------------------
Total 답변 개수: 70


In [10]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
answer_text_list = []

for ans_url in tqdm(ans_url_list):
    driver.get(ans_url)
    time.sleep(1)
    more = driver.find_element_by_xpath('//*[@id="nextPageButton"]')
    disp = more.get_attribute('style')      # 더보기 할 답변이 남아있지 않은 경우 "style:None" 이 존재한다
    try:
        # 더보기 할 답변이 남지 않을 때까지 while문을 돌린다
        while disp:     
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(1)
    except:
        time.sleep(1)
    answers = driver.find_elements_by_class_name('answer-content__item._contentWrap._answer')
    for ans in answers:
        ps = ans.find_elements_by_tag_name('p')
        for p in ps:
            answer_text_list.append(p.text)
driver.close()

100%|██████████| 21/21 [01:14<00:00,  3.56s/it]


In [11]:
len(answer_text_list)

1694

### 원본 저장(전처리 X)

In [12]:
df = pd.DataFrame({
    'answer': answer_text_list
})
df.head()

,answer
0,xogh**** 님 답변
1,2022년 차기 대통령 선거 일정이
2,3월로 예정이 되면서 이에 따라
3,대선 후보주자들 중 이재명은
4,유력 후보주자 중 한명입니다.


In [13]:
start = start_date.strftime("%y%m%d")
end = datetime.today().strftime("%y%m%d")
print(start, end)

210417 210420


In [14]:
df.to_csv(f'./data/{KEYWORD}_{start}_to_{end}_org.txt', sep='\t', index=False)

### 데이터 전처리

In [15]:
df_test = pd.read_table(f'./data/{KEYWORD}_{start}_to_{end}_org.txt')
df_test.head()

,answer
0,xogh**** 님 답변
1,2022년 차기 대통령 선거 일정이
2,3월로 예정이 되면서 이에 따라
3,대선 후보주자들 중 이재명은
4,유력 후보주자 중 한명입니다.


In [16]:
df_test.shape

(1679, 1)

In [17]:
# 공백 여부 확인
drop_idx = df_test[df_test.answer == ""].index
print(len(drop_idx), drop_idx)

0 Int64Index([], dtype='int64')


In [18]:
# null값 여부 확인
df_test.isnull().sum()

answer    472
dtype: int64

In [19]:
# null값을 가지고 있는 행 제거
df_test.dropna(axis=0, inplace=True)
df_test.shape

(1207, 1)

In [20]:
df_test.head()

,answer
0,xogh**** 님 답변
1,2022년 차기 대통령 선거 일정이
2,3월로 예정이 되면서 이에 따라
3,대선 후보주자들 중 이재명은
4,유력 후보주자 중 한명입니다.


In [21]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 1207


In [22]:
df_test.to_csv(f'./result/{KEYWORD}_{start}_to_{end}_res.txt', sep='\t', index=False)